### 1. Notwendige Bibliotheken müssen am Anfang installiert werden
python 3.9.6 wird empfohlen.

In [ ]:
!pip3 install paho-mqtt


### 2. Notwendige Bibliotheken und Dummy Daten werden importiert.


In [1]:
import time
import json
import time
import subprocess
import paho.mqtt.client as mqtt

mainPath = "./dumy_data/"


In [80]:
# Open log file and read messages
with open(mainPath + "spat.json", "r") as spat_file:
    lines_spat = spat_file.readlines()
with open(mainPath + "map.json", "r") as map_file:
    lines_map = map_file.readlines()
with open(mainPath + "gps.json", "r") as gps_file:
    lines_gps = gps_file.readlines()

### 3. Broker einrichten und starten (mosquitto muss davor installiert werden https://mosquitto.org/download/)

In [11]:
mosquitto_process = subprocess.Popen(["mosquitto"])

1686775307: mosquitto version 2.0.15 starting
1686775307: Using default config.
1686775307: Starting in local only mode. Connections will only be possible from clients running on this machine.
1686775307: Create a configuration file which defines a listener to allow remote access.
1686775307: For more details see https://mosquitto.org/documentation/authentication-methods/
1686775307: Opening ipv4 listen socket on port 1883.
1686775307: Opening ipv6 listen socket on port 1883.
1686775307: mosquitto version 2.0.15 running


### 4. Client einrichten und mit Broker verbinden. Dummy Daten werden von diesem Client veröffentlicht

In [82]:
def on_connect(client, userdata, flags, rc):
    print(f"Connected with result code {rc}")
    client.subscribe("your/topic")

def on_message(client, userdata, msg):
    print(f"Received message: {msg.topic} {msg.payload.decode()}")

# def on_publish(client, userdata, mid):
    # print(f"Message published (mid: {mid})")

# Set up MQTT client
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message
# client.on_publish = on_publish

client.connect("localhost", 1883)
client.loop_start()

Connected with result code 0


1686805404: New connection from ::1:54370 on port 1883.
1686805404: New client connected from ::1:54370 as auto-94C99A8E-3B8D-DC05-9362-469735213367 (p2, c1, k60).


### 4. Alle Daten bei jeweiligem Topic veröffentlichen. (Dummy Broker wird gerstartet)

In [83]:
# Send each messages to MQTT broker
rate = 2
spatTopic = "v2x-uca/output/json/spat"
mapTopic = "v2x-uca/output/json/map"
gpsTopic = "v2x/rx/obu_gnss"

for line_spat, line_map, line_gps in zip(lines_spat, lines_map, lines_gps):

    # extract spat, map and gps
    spat_msg = json.loads(line_spat)
    map_msg = json.loads(line_map)
    gps_msg = json.loads(line_gps)

    # publish spat, map and gps
    client.publish(spatTopic, json.dumps(spat_msg))
    client.publish(mapTopic, json.dumps(map_msg))
    client.publish(gpsTopic, json.dumps(gps_msg))

    # sleep
    time.sleep(rate)



1686805491: Client auto-0B183C84-5DF5-9501-44E8-F18565D07E7D closed its connection.
1686805508: New connection from 127.0.0.1:54421 on port 1883.
1686805508: New client connected from 127.0.0.1:54421 as auto-7DB36EB2-68CA-385E-F51C-5EC46DB8C376 (p2, c1, k60).
1686805631: Client auto-7DB36EB2-68CA-385E-F51C-5EC46DB8C376 closed its connection.


KeyboardInterrupt: 

Um alle gepushlished Nachrichten zu sehen, hat man 2 Möglichkeiten:
<br>
<br>
Alternative 1. Einen lokalen mosquitto client kann im Terminal gestartet werden. 
<br>
  1.1. mosquitto muss davor installiert werden.<br>
  2.2. Im Terminal `mosquitto_sub -h localhost -t "#"` ausführen 
<br>
<br>

Alternative 2. Docker container `nodered`.
<br>
  2.1. `Docker` und `Docker Compose` müssen davor installiert werden.<br>
  2.2. Im Terminal `docker compose --profile development up nodered` ausführen.<br>
  2.3. Im browser http://localhost:1880 öffnen. 

### 5. Clean up

In [84]:
# Disconnect from MQTT broker
client.disconnect()
client.loop_stop()

1686805727: Client auto-94C99A8E-3B8D-DC05-9362-469735213367 disconnected.


In [85]:
# stop the MQTT broker
mosquitto_process.terminate()

1686805728: mosquitto version 2.0.15 terminating
